## Importing Libs

In [17]:
%pip install pyLDAvis

StatementMeta(, 25, -1, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 60.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.4/381.4 KB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 110.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 82.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 KB 68.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Not uninstalling numpy at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-a35ac763-645d-4e0b-b67f-057694b36a67
    Can't uninstall 'numpy'. No files were found to uninstall.
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.1
    Not uninstalling pandas at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-a35ac763-645d-4e0b-b67f-057694b36a67
    Can't u

In [18]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType,StructField, StringType


StatementMeta(synsp03, 25, 28, Finished, Available)

In [19]:
root = 'abfss://default@stdatalakeakita.dfs.core.windows.net/synapse/workspaces/syn-synfactoreddatathon01-dev'

df = spark.read\
.format('delta')\
.load(f'{root}/gold/gold_amazon_reviews_automotive_text_processed')

df_pandas = df.pandas_api()

StatementMeta(synsp03, 25, 29, Finished, Available)

'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


#### 1. Loading TF and IDF models

In [20]:
from pyspark.ml.feature import IDFModel, CountVectorizerModel
tf_model = CountVectorizerModel.load(f'{root}/models/vectorizer/count-vectorizer-model')
idf_model = IDFModel.load(f'{root}/models/idf/idf-model')

df_tf = tf_model.transform(df)
df_tfidf = idf_model.transform(df_tf)

StatementMeta(synsp03, 25, 30, Finished, Available)

#### 2. Training LDA

In [46]:
from pyspark.ml.clustering import LDA
lda = LDA(k=15, seed=1, optimizer="online", featuresCol='tf_idf_features')
lda.setMaxIter(70)
lda.clear(lda.maxIter)
model = lda.fit(df_tfidf)
model.setSeed(1)

StatementMeta(synsp03, 25, 56, Submitted, Running)

#### 3. Save and Load Model

In [ ]:
from pyspark.ml.clustering import LocalLDAModel
lda_path = f'{root}/models/lda' + "/lda_k15"
lda.save(lda_path)
lda_model_path = f'{root}/models/lda' + "/lda_model_20230108_k15"
model.save(lda_model_path)

modelLoaded = LocalLDAModel.load(f'{root}/models/lda/lda_model_20230108_k15')
df_reviews_automotiveModel = modelLoaded.transform(df_tfidf)


StatementMeta(, , , Waiting, )

In [33]:
from pyspark.ml.functions import vector_to_array

display(df_reviews_automotiveModel.where(
    F.array_contains(
        vector_to_array(F.col('topicDistribution')), F.lit(0)) == F.lit(True)))

StatementMeta(synsp03, 25, 43, Finished, Available)

SynapseWidget(Synapse.DataFrame, fda9d6a0-5c3b-4276-b621-178f01021d10)

In [ ]:
df_reviews_automotiveModel_2 = df_reviews_automotiveModel.where(
    F.array_contains(
        vector_to_array(F.col('topicDistribution')), F.lit(0)) == F.lit(False))

StatementMeta(, , , Waiting, )

#### 5. Visualize results

In [ ]:
import numpy as np
topic_term_dists = np.array(modelLoaded.topicsMatrix().toArray()).T
print(topic_term_dists)

StatementMeta(, , , Waiting, )

In [ ]:
doc_topic_dists = np.array(
    [x.toArray() for x in df_reviews_automotiveModel_2.select(["topicDistribution"]).toPandas()['topicDistribution']]
)
print(doc_topic_dists)

StatementMeta(, , , Waiting, )

In [ ]:
doc_lengths = [x[0] for x in df_reviews_automotiveModel_2.select(F.size(df_reviews_automotiveModel_2.finished_lemma)).collect()]
print(doc_lengths)

StatementMeta(, , , Waiting, )

In [ ]:
counts = df_reviews_automotiveModel_2.select((F.explode(df_reviews_automotiveModel_2.finished_lemma)).alias("tokens")).groupby("tokens").count()
wc = {i['tokens']: i['count'] for i in counts.collect()}
wc = [wc[x] for x in tf_model.vocabulary]
term_frequency = wc

print(term_frequency)

StatementMeta(, , , Waiting, )

In [ ]:
import pyLDAvis
vis = pyLDAvis.prepare(vocab = tf_model.vocabulary, 
                       topic_term_dists=topic_term_dists,
                       doc_topic_dists=doc_topic_dists,
                       doc_lengths=doc_lengths,
                       term_frequency=term_frequency
)

StatementMeta(, , , Waiting, )

In [45]:
pyLDAvis.display(vis)

StatementMeta(synsp03, 25, 55, Finished, Available)

#### With 15 topics

In [ ]:
pyLDAvis.display(vis)